In [ ]:
!pip install --upgrade langchain 

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_template("{topic}에 대한 간단한 농담을 말해줘.")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser # LCEL의 기본 파이프라인

chain.invoke({"topic": "아이스크림"})

In [ ]:
!pip install langchain docarray tiktoken

In [ ]:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["해리슨은 학교에서 일합니다.", "곰은 꿀을 좋아해"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """다음 지문에만 근거해서 질문에 답하세요:
{context}

질문: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel( # 여러개의 Runnable을 병렬적으로 실행
    {"context": retriever, "question": RunnablePassthrough()} # RunnablePassthrough는 값을 입력받아 그대로 전달하는 객체
)
chain = setup_and_retrieval | prompt | model | output_parser

# retrieval | prompt | model | output. RAG의 기본 파이프라인

In [ ]:
chain.invoke("해리슨은 어디에서 일하나요?")

In [ ]:

chain.invoke("곰은 무엇을 좋아하나요?")

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    "{topic}에 대한 간단한 농담을 해줘."
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-3.5-turbo")
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

In [ ]:
chain.batch(["아이스크림", "스파게티", "만두"])

# stream: 입력에 대해 청크를 스트리밍
# invoke: 입력에 대해 체인을 호출
# batch: 입력목록에 대해 체인을 배치로 호출

In [ ]:
await chain.ainvoke("ice cream")

# astream: 비동기로 입력에 대해 청크를 스트리밍
# ainvoke: 비동기로 입력에 대해 체인을 호출
# abatch: 비동기로 입력목록에 대해 체인을 배치로 호출

In [ ]:
await chain.abatch(
    ["아이스크림", "스파게티", "만두"]
)

In [ ]:
chain.input_schema.schema()

In [ ]:
chain.output_schema.schema()